<h2>ASSIGNMENT 4: bayesian network from scratch<h2>

<h4><b>Author</b>: Aliprandi Francesco<h4>


<h4><b>1.1 Class Node</b></h4>

In [129]:
import numpy as np
import random

class Node:   
    def __init__(self, name, values, parents, table):
        self.name: str = name
        self.values: list = values
        self.parents: list = parents
        #to ensure parents sorted in CPTs
        if len(self.parents) != 0:
            self.table: dict = {tuple(sorted(k)): v for k, v in table.items()}
        else:
            self.table: dict = table
    
    def get_name(self):
        return self.name
    
    def get_values(self):
        return self.values
    
    def get_parents(self):
        return self.parents
    
    def get_table(self):
        return self.table
    
    def sample_value(self, parent_values):
        #if node has no parents
        if(len(parent_values) == 0):
            values = list(self.table.keys())
            prob = list(self.table.values())
            return np.random.choice(values, p=prob)
        #if node has parents and it's binomial
        elif len(self.values) == 2:
            values = ["True", "False"]
            key = tuple(sorted(parent_values))
            prob = self.table[key]
            prob = [prob, 1-prob]
            return np.random.choice(values, p=prob)
                
                
    def get_sample(self, sampling):
        #if node has no parents
        if(len(self.parents) == 0):
            return self.sample_value([])
        #if node has parents
        parents_value = []
        for node in sampling.keys():
            for parent in self.parents:
                if node == parent.get_name():
                    parents_value.append(node + "_" + sampling[node])
        return self.sample_value(parents_value)
               
        

<h4><b>1.2 Class Network</b></h4>

In [130]:
class Network:
    def __init__(self, nodes):
        self.nodes: list = nodes
    
    def add_node(self, node):
        self.nodes.append(node)
        
    def get_nodes(self):    
        return self.nodes
    
    def topological_sort(self):
        visited = set()
        stack = []

        def dfs(node):
            visited.add(node)
            for parent in node.parents:
                if parent not in visited:
                    dfs(parent)
            stack.insert(0,node)
    
        for node in self.nodes:
            if node not in visited:
                dfs(node)
        return stack[::-1]
        
    
    def ancestral_sampling(self):
        sampling = {}
        random.shuffle(self.nodes)
        sorted_graph = self.topological_sort()
        for node in sorted_graph:
            value = node.get_sample(sampling)
            sampling[node.get_name()] = value          
        return sampling

<h4><b>1.3 Bayesian network creation</b></h4>

Creation of the bayesian network that models the ignition of an old car, according to the following graph

<img src="./image/bnet.png" alt="car ignition network" width=500 height=250>



In [131]:
alternator = Node("Alternator", ["New", "Worn","broken"], [], {"New": 0.3, "Worn": 0.55, "Broken": 0.15})
fanbelt = Node("Fanbelt", ["True", "False"], [], {"True": 0.69, "False": 0.31})
starter = Node("Starter", ["True", "False"], [], {"True": 0.77, "False": 0.23})
sparks = Node("Sparks", ["True", "False"], [], {"True": 0.78, "False": 0.22})
fuel_pump = Node("Fuel_pump", ["True", "False"], [], {"True": 0.77, "False": 0.23})
fuel_level = Node("Fuel_level", ["Low", "Normal", "High"], [], {"Low": 0.1, "Normal": 0.8, "High": 0.1})

battery = Node("Battery", ["True", "False"], [alternator, fanbelt], {("Alternator_New","Fanbelt_True"): 0.85, ("Alternator_New","Fanbelt_False"): 0.27, ("Alternator_Worn","Fanbelt_True"): 0.73, ("Alternator_Worn","Fanbelt_False"): 0.24, ("Alternator_Broken","Fanbelt_True"): 0.52, ("Alternator_Broken","Fanbelt_False"): 0.11})
dashboard = Node("Dashboard", ["True","False"], [battery, starter], {("Battery_True","Starter_True"): 0.92, ("Battery_True","Starter_False"): 0.15, ("Battery_False","Starter_True"): 0.2, ("Battery_False","Starter_False"): 0.08})
fuel_subsystem = Node("Fuel_subsystem", ["True", "False"], [fuel_pump, fuel_level], {("Fuel_level_High", "Fuel_pump_True"): 0.88, ("Fuel_level_High", "Fuel_pump_False"): 0.12, ("Fuel_level_Normal", "Fuel_pump_True"): 0.9, ("Fuel_level_Normal", "Fuel_pump_False"): 0.18, ("Fuel_level_Low", "Fuel_pump_True"): 0.79, ("Fuel_level_Low", "Fuel_pump_False"): 0.11})
engine = Node("Engine", ["True", "False"], [dashboard, sparks, fuel_subsystem], {("Dashboard_True", "Fuel_subsystem_True", "Sparks_True"): 0.92, ("Dashboard_True", "Fuel_subsystem_True", "Sparks_False"): 0.04, ("Dashboard_True", "Fuel_subsystem_False", "Sparks_True"): 0.38, ("Dashboard_True", "Fuel_subsystem_False", "Sparks_False"): 0.02, ("Dashboard_False", "Fuel_subsystem_True", "Sparks_True"): 0.44, ("Dashboard_False", "Fuel_subsystem_True", "Sparks_False"): 0.03, ("Dashboard_False", "Fuel_subsystem_False", "Sparks_True"): 0.07, ("Dashboard_False", "Fuel_subsystem_False", "Sparks_False"): 0.01})


In [132]:
car = Network([alternator, fanbelt, starter, sparks, fuel_pump, fuel_level, battery, dashboard, fuel_subsystem, engine])

c = 0
for i in range(100000):
    sampling = car.ancestral_sampling()
    if(sampling["Engine"] == "True"):
        c+=1
print("car starts", c/100000*100, "% of the time")


car starts 43.718 % of the time
